In [ ]:
import sys
ros_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'
if ros_path in sys.path:
    sys.path.remove(ros_path)
import cv2
sys.path.append('/opt/ros/kinetic/lib/python2.7/dist-packages')

import pyzed.sl as sl
import math
import numpy as np

In [ ]:
def check_episode(distance, episode_start, episode_end):
    init_distance = 5.0
    final_distance = 1.0
    
    if ~np.isnan(distance):
        if distance <= 5.0 and distance >= 1.0:
            episode_start = True
            episode_end = False        
        else:
            episode_start = False
            episode_end = True        
    
    return episode_start, episode_end    

In [ ]:
def main():
    global a
    # Create a Camera object
    zed = sl.Camera()

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters()
    init_params.depth_mode = sl.DEPTH_MODE.PERFORMANCE  # Use PERFORMANCE depth mode
    init_params.coordinate_units = sl.UNIT.METER  # Use meter units (for depth measurements)
    init_params.camera_resolution = sl.RESOLUTION.HD720

    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        exit(1)

    # Create and set RuntimeParameters after opening the camera
    runtime_parameters = sl.RuntimeParameters()
    runtime_parameters.sensing_mode = sl.SENSING_MODE.STANDARD  # Use STANDARD sensing mode
    # Setting the depth confidence parameters
    runtime_parameters.confidence_threshold = 100
    runtime_parameters.textureness_confidence_threshold = 100

    # Capture 50 images and depth, then stop
    i = 0
    image = sl.Mat()
    depth = sl.Mat()
    point_cloud = sl.Mat()
    cam = sl.Camera()
    
    mirror_ref = sl.Transform()
    mirror_ref.set_translation(sl.Translation(2.75,4.0,0))
    tr_np = mirror_ref.m
    
    episode = 0
    image_buffer = []
    
    episode_start = False
    episode_end = True
    
    while True:
        # A new image is available if grab() returns SUCCESS
        if zed.grab(runtime_parameters) == sl.ERROR_CODE.SUCCESS:            
            # Retrieve left image
            zed.retrieve_image(image, sl.VIEW.LEFT)
            # Retrieve depth map. Depth is aligned on the left image
            zed.retrieve_measure(depth, sl.MEASURE.DEPTH)
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA)
            
            a = image.get_data()
            # Visualization of point cloud map
            "If you want to see the shape of the point cloud data, use: point_cloud.get_data()"
            # See point cloud data            
#             cv2.imshow('Point_cloud', point_cloud.get_data())
#             cv2.waitKey(1)
            
            # see depth image
            cv2.imshow("Depth", depth.get_data())
            
            # See rgb image
            cv2.imshow("RGB", image.get_data())
            cv2.waitKey(1)                                    
                        
            # Get and print distance value in mm at the center of the image
            # We measure the distance camera - object using Euclidean distance
            distance_matrix = []
            for x in range(320,960,10):
                for y in range(180,540,10):
                    x = round(image.get_width() / 2)
                    y = round(image.get_height() / 2)
                    err, point_cloud_value = point_cloud.get_value(x, y)
                    distance = math.sqrt(point_cloud_value[0] * point_cloud_value[0] +
                                         point_cloud_value[1] * point_cloud_value[1] +
                                         point_cloud_value[2] * point_cloud_value[2])
            
                    if not np.isnan(distance) and not np.isinf(distance):
                        distance_matrix.append(distance)
            
            episode_start, episode_end = check_episode(np.mean(distance_matrix), episode_start, episode_end)
            if episode_start and not episode_end:
                print('recording. \n')
            else:
                print('Not recording \n')
            
#             if episode_flag:
#                 "Store the data"
#                 np.save('./training_data/RGB_images/epsisode_'+str(episode)+'.npy', image_buffer)                

#                 episode_flag = False
#                 image_buffer = []
#                 episode = episode + 1
#             else:
#                 "Append the image data into a matrix"
#                 image_buffer.append(a[:,:,:3])
#                 episode_flag = False
                                
            
            if not np.isnan(np.mean(distance_matrix)) and not np.isinf(np.mean(distance_matrix)):
                print("Distance to camera:{}".format(np.mean(distance_matrix)), end='\r')            
                        
                
            sys.stdout.flush()

    # Close the camera
    zed.close()

In [ ]:
if __name__ == "__main__":
    main()